# How to use Anyscale function calling with LLMs
Function calling extends the power capabilities of LLMs. It allolws you to format
the output of an LLM response into a JSON object, which then can be fed down stream
to an actual function as a argument to process the response.

OpenAI documention states the basic steps involved in function calling: 

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

<img src="./images/gpt_function_calling.png">

Let's first demonstrate how we use this feature. Let's first specify a function and use the API to generate function arguments.


In [17]:
import warnings
import os
import json

import openai
from openai import OpenAI

from pydantic import BaseModel, Field
from dotenv import load_dotenv, find_dotenv
from typing import Dict, Any, List

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints.

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
google_api_key = os.getenv("GOOGLE_API_KEY", "")
weather_api_key = os.getenv("WEATHER_API_KEY", "")

MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=mistralai/Mistral-7B-Instruct-v0.1; base=https://api.endpoints.anyscale.com/v1


In [3]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

## Simple case: Example 1: Query the Anyscale endpoint with Mistral 

Since Anyscale endpoint supports JSON response format, let's use
Pydantic validation and base class for our response object

In [4]:
class QueryResponse(BaseModel):
    finalist_team: str = Field(description="World Cup finalist team"),
    winning_team: str = Field(description="World Cup final winner team"),
    final_score: str = Field(description="Final score")

Let'send a basic query and see what result we get and its JSON format

In [5]:
response = chat_completion = client.chat.completions.create(
    model=MODEL,
    response_format={
        "type": "json_object", 
        "schema": QueryResponse.schema_json()
    },
    messages=[
        {"role": "system", 
         "content": "You are the FIFA World Cup Soccer assistant designed to output JSON for queries"},
        {"role": "user", 
         "content": """What national teams played in the FIFA World Cup Soccer 1998 Finals played 
                        in Francce, who was the winner and who was the loser?"""
        }
    ],
    temperature=0.0
)

In [6]:
result = response.choices[0].message.content

#### Convert the JSON response into a dictonary
This output matches JSON format schema defined above using Pydantic QueryResponse class

In [7]:
import json

result = response.choices[0].message.content
# Convert string into a dictionary
json_schema = json.loads(result)

for k, v in json_schema.items():
    print(f"{k}:{v}")

winning_team:France
final_score:3-0
finalist_team:Brazil


In [8]:
# Convert dictionary into a printable JSON string
print(json.dumps(json_schema, indent=2))

{
  "winning_team": "France",
  "final_score": "3-0",
  "finalist_team": "Brazil"
}


## Example 2: Query the Anyscale endpoint with Mistral 


In [9]:
from typing import List
class MathResponse(BaseModel):
    prime_numbers: List[int] = Field(description="List of prime numbers between 2 and 100")
    sum: int = Field(description="Sum of the all the prime numbers between 2 and 100")

In [10]:
response = chat_completion = client.chat.completions.create(
    model=MODEL,
    response_format={
        "type": "json_object", 
        "schema": MathResponse.schema_json()
    },
    messages=[
        {"role": "system", 
         "content": """You are Math tutor who can answer simple math problems and return 
         reponse in JSON format"""
        },
        {"role": "user", 
        "content": """Generate a list of all prime numbers between 2 and 100 and add the
        numbers in the list"""
        }
    ],
    temperature=0.7
)

In [11]:
import json

result = response.choices[0].message.content
# Convert string into a dictionary
json_schema = json.loads(result)

for k, v in json_schema.items():
    print(f"{k}:{v}")

sum:12
prime_numbers:[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [12]:
# Convert dictionary into a printable JSON string
print(json.dumps(json_schema, indent=2))

{
  "sum": 12,
  "prime_numbers": [
    2,
    3,
    5,
    7,
    11,
    13,
    17,
    19,
    23,
    29,
    31,
    37,
    41,
    43,
    47,
    53,
    59,
    61,
    67,
    71,
    73,
    79,
    83,
    89,
    97
  ]
}


## Example 2: Query the Anyscale endpoint with Mistral 


In [13]:
from typing import Tuple
class FractionResponse(BaseModel):
    fractions: List[Tuple[int, int]] = Field(description="List of unique fractrions between 1 and 5 as Python tuples")
    sum: int = Field(description="Sum of the all unique fractrions between 1 and 5")
    common_denominator: int = Field(description="The common denominator among the unique fractions between 1 and 5")

In [14]:
response = chat_completion = client.chat.completions.create(
    model=MODEL,
    response_format={
        "type": "json_object", 
        "schema": FractionResponse.schema_json()
    },
    messages=[
        {"role": "system", 
         "content": """You are Math tutor who can answer simple math problems and return 
         reponse in JSON format"""
        },
        {"role": "user", 
        "content": """Generate a list of all unique fractions between between 1 and 5 as Python tuples,  
        add the tuple fractions in the list, and find their common denominator"""
        }
    ],
    temperature=0.7
)

In [15]:
import json

result = response.choices[0].message.content
# Convert string into a dictionary
json_schema = json.loads(result)

for k, v in json_schema.items():
    print(f"{k}:{v}")

sum:25
fractions:[[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 8], [1, 10], [1, 12], [1, 15], [1, 20], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [2, 12], [2, 15], [2, 20], [3, 4], [3, 5], [3, 6], [3, 8], [3, 10], [3, 12], [3, 15], [3, 20], [4, 5], [4, 6], [4, 8], [4, 10], [4, 12], [4, 15], [4, 20], [5, 6], [5, 8], [5, 10], [5, 12], [5, 15], [5, 20]]
common_denominator:60


In [16]:
# Convert dictionary into a printable JSON string
print(json.dumps(json_schema, indent=2))

{
  "sum": 25,
  "fractions": [
    [
      1,
      2
    ],
    [
      1,
      3
    ],
    [
      1,
      4
    ],
    [
      1,
      5
    ],
    [
      1,
      6
    ],
    [
      1,
      8
    ],
    [
      1,
      10
    ],
    [
      1,
      12
    ],
    [
      1,
      15
    ],
    [
      1,
      20
    ],
    [
      2,
      3
    ],
    [
      2,
      4
    ],
    [
      2,
      5
    ],
    [
      2,
      6
    ],
    [
      2,
      7
    ],
    [
      2,
      8
    ],
    [
      2,
      9
    ],
    [
      2,
      10
    ],
    [
      2,
      12
    ],
    [
      2,
      15
    ],
    [
      2,
      20
    ],
    [
      3,
      4
    ],
    [
      3,
      5
    ],
    [
      3,
      6
    ],
    [
      3,
      8
    ],
    [
      3,
      10
    ],
    [
      3,
      12
    ],
    [
      3,
      15
    ],
    [
      3,
      20
    ],
    [
      4,
      5
    ],
    [
      4,
      6
    ],
    [
      4,
      8
   

## How to generate function arguments
For Anyscale endpoints, the idea of function calling is not that different from 
OpenAI's. Basically, according to [Anyscale function calling blog](https://www.anyscale.com/blog/anyscale-endpoints-json-mode-and-function-calling-features), it boils down to simple steps:
 1. You send a query with functions and parameters to the LLM.
 2. The LLM decides to either use a function or not.
 3. If not using a function, it replies in plain text, providing an answer or asking for more information.
 4. If using a function, it recommends an API and gives usage instructions in JSON.
 5. You execute the API in your application.
 6. Send the API's results back to the LLM.
 7. The LLM analyzes these results and guides the next steps.

#### Example 1: Extract the generated arguments

Let's process generated arguments to plot a map of cities where we hold 
Ray meetups. These generated satellite coordinates, generated as a JSON object
by the LLM, are fed into a function to generate an HTML file and map markers of
each city.

The idea is here is to nudge the LLM to generate JSON object, which can be easily converted into a Python dictionary as an argument to a function downstream to create the HTML and render a map.

In [17]:
tools = [ 
        {
            "type": "function",
            "function": {
                "name": "generate_ray_meetup_map",
                "description": "Generate HTML map for global cities where Ray meetups are hosted",
                "parameters": {
                    "type": "object",
                    "properties": {
                                "location": {
                                    "type": "string",
                                    "description": "The city name e.g., San Francisco, CA",
                                    },
                                "latitude": {
                                    "type": "string",
                                    "description": "Latitude satelite coordinates",
                                    },
                                "longitude": {
                                    "type": "string",
                                    "description": "Longitude satelite coordinates",
                                    },
                                }
        
                            }
                },
                "required": ["location, latitude, longitude"]
        }
]

In [18]:
def generate_city_map_completion(clnt: object, model: str, user_content:str) -> object:
    chat_completion = clnt.chat.completions.create(
        model=model,
        messages=[ {"role": "system", "content": f"You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
                   {"role": "user", "content": user_content}],
        tools=tools,
        tool_choice="auto",
        temperature=0.7)
        
    response = chat_completion.choices[0].message
    return response

In [19]:
user_content = """Generate satelite coordinates latitude and longitude for location of San Francisco where Ray meetup is hosted 
at Anyscale Headquaters on 55 Hawthorne Street, San Francisco, CA 94105
"""

In [20]:
print(f"Using Endpoints: {openai.api_base} ...\n")
response_1 = generate_city_map_completion(client, MODEL, user_content)
print(response_1)

Using Endpoints: https://api.endpoints.anyscale.com/v1 ...

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_35faf175548046b28cd86d9b1080ccf8', function=Function(arguments='{"location": "San Francisco", "latitude": "37.7749", "longitude": "-122.4194"}', name='generate_ray_meetup_map'), type='function')])


In [21]:
json_arguments = response_1.dict()
json_arguments

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'call_35faf175548046b28cd86d9b1080ccf8',
   'function': {'arguments': '{"location": "San Francisco", "latitude": "37.7749", "longitude": "-122.4194"}',
    'name': 'generate_ray_meetup_map'},
   'type': 'function'}]}

In [22]:
# Extract specific items from the dict
func_name = json_arguments['tool_calls'][0]['function']['name']
print(f"Function name: {func_name}")

Function name: generate_ray_meetup_map


In [23]:
funcs = json_arguments['tool_calls'][0]['function']['arguments']
funcs_args = json.loads(funcs)
print(f"Arguments: {funcs_args}")

Arguments: {'location': 'San Francisco', 'latitude': '37.7749', 'longitude': '-122.4194'}


In [24]:
html_file_path = './world_map_nb_func_anyscale_with_cities.html'

In [25]:
import folium

def generate_ray_meetup_map(coordinates: Dict[str, Any]) -> None:
    # Create a base map
    m = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)
    # Adding markers for each city
    folium.Marker([coordinates["latitude"], coordinates["longitude"]], popup=coordinates["location"]).add_to(m)
    # Display the map
    m.save(html_file_path)

#### Invoke the function from within our notebook.
This is our downstream function being invoked with the extract arguments
from the JSON response generated by the LLM.

In [26]:
from IPython.display import IFrame

# Assuming the HTML file is named 'example.html' and located in the same directory as the Jupyter Notebook
generate_ray_meetup_map(funcs_args)

# Display the HTML file in the Jupyter Notebook
IFrame(src=html_file_path, width=700, height=400)

#### Example 2: Generate function arguments

In [6]:
def get_weather_data_for_cities(params:Dict[Any, Any]=None,
                    api_base:str="http://api.weatherstack.com/current") -> Dict[str, str]:
    """
    Retrieves weather data from the OpenWeatherMap API for cities
    """
    import requests
    url = f"{api_base}"
    response = requests.get(url, params=params)
    return response.json()

In [7]:
tools_2 = [
    {
        "type": "function",
        "function": {
            "name": "get_weather_data_for_cities",
            "description": "Get the current weather forecast in the cities",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "array",
                        "items": {
                                "type": "string",
                                "description": "The list of city names e.g., San Francisco, New York, London"
                            }
                    }
                }
             }
        },
        "required": ["query"]
    }
]

In [8]:
def generate_weather_forecast_completion(clnt: object, model: str, user_content:str) -> object:
    chat_completion = clnt.chat.completions.create(
        model=model,
        messages=[ {"role": "system", "content": f"You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
                   {"role": "user", "content": user_content}],
        tools=tools_2,
        tool_choice="auto",
        temperature=0.7)
        
    response = chat_completion.choices[0].message
    return response

In [9]:
user_content = """
                Generate weather forecast and temperature for 
                London, New York, San Franciso for today.
                """

In [10]:
print(f"Using Endpoints: {openai.api_base} ...\n")
print(f"Using model: {MODEL} ...\n")
weather_response = generate_weather_forecast_completion(client, MODEL, user_content)
print(weather_response)

Using Endpoints: https://api.endpoints.anyscale.com/v1 ...

Using model: mistralai/Mistral-7B-Instruct-v0.1 ...

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ef44da1461bd4d709272442cb5974877', function=Function(arguments='{"query": ["London", "New York", "San Francisco"]}', name='get_weather_data_for_cities'), type='function')])


In [12]:
json_arguments = weather_response.dict()
json_arguments

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'call_ef44da1461bd4d709272442cb5974877',
   'function': {'arguments': '{"query": ["London", "New York", "San Francisco"]}',
    'name': 'get_weather_data_for_cities'},
   'type': 'function'}]}

In [15]:
# Extract specific items from the dict
func_name = json_arguments['tool_calls'][0]['function']['name']
print(f"Function name: {func_name}")

Function name: get_weather_data_for_cities


In [30]:
funcs = json_arguments['tool_calls'][0]['function']['arguments']
funcs_args = json.loads(funcs)
print(f"Arguments: {funcs_args}")

Arguments: {'query': ['London', 'New York', 'San Francisco']}


#### Invoke the function from within our notebook.
This is our downstream function being invoked with the extracted arguments
from the JSON response generated by the LLM.

In [61]:
# Iterate over cities since we have a list
cities = funcs_args['query']
weather_statements = []
for city in cities:
    params = {'query': city,
              'access_key': weather_api_key,
              'units': "f"
             }
    weather_data = get_weather_data_for_cities(params)
    c = f"The weather and forecast in the {city} is {weather_data['current']['temperature']} F and {weather_data['current']['weather_descriptions'][0]}"
    weather_statements.append(c) 
weather_statements

['The weather and forecast in the London is 46 F and Partly cloudy',
 'The weather and forecast in the New York is 39 F and Overcast',
 'The weather and forecast in the San Francisco is 55 F and Partly cloudy']

In [64]:
paragraph = ' '.join(weather_statements)

### Let LLM generate the response for the orignal request

In [78]:
messages=[ {"role": "system", "content": f"You are a helpful assistant for weather forecast. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
            {"role": "user", "content": user_content}]
messages.append(json_arguments)
messages

[{'role': 'system',
  'content': "You are a helpful assistant for weather forecast. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': '\n                Generate weather forecast and temperature for \n                London, New York, San Franciso for today.\n                '},
 {'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_ef44da1461bd4d709272442cb5974877',
    'function': {'arguments': '{"query": ["London", "New York", "San Francisco"]}',
     'name': 'get_weather_data_for_cities'},
    'type': 'function'}]}]

In [79]:
messages.append(
    {"role": "tool",
     "tool_call_id": json_arguments['tool_calls'][0]['id'],
     "name": json_arguments['tool_calls'][0]['function']["name"],
     "content": paragraph}
)
messages

[{'role': 'system',
  'content': "You are a helpful assistant for weather forecast. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': '\n                Generate weather forecast and temperature for \n                London, New York, San Franciso for today.\n                '},
 {'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_ef44da1461bd4d709272442cb5974877',
    'function': {'arguments': '{"query": ["London", "New York", "San Francisco"]}',
     'name': 'get_weather_data_for_cities'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_ef44da1461bd4d709272442cb5974877',
  'name': 'get_weather_data_for_cities',
  'content': 'The weather and forecast in the London is 46 F and Partly cloudy The weather and forecast in the New York is 39 F and Overcast The weather and forecast in the San Francisco is 55 F and Partly cloud

In [82]:
chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools_2,
    tool_choice="auto",
    temperature=0.7)
        
response = chat_completion.choices[0].message
print(response.content)

 The weather and forecast in London is 46 F and Partly cloudy, the weather and forecast in New York is 39 F and Overcast, and the weather and forecast in San Francisco is 55 F and Partly cloudy.
